## Manage Dataproc Workflows using gcloud Commands
Let us see how we can manage Dataproc Workflows using gcloud commands.
* Step 1: Create Dataproc Workflow Template
* Step 2: Configure active Dataproc cluster (we can also configure new cluster)
* Step 3: Add Spark SQL or Pyspark Jobs to Dataproc Workflow Templates with Dependencies
* Step 4: Run and Validate the Dataproc Workflow Template

We can take care of all the steps using `gcloud` commands.

In [5]:
!gsutil ls -r gs://airetail/scripts

gs://airetail/scripts/:

gs://airetail/scripts/daily_product_revenue/:
gs://airetail/scripts/daily_product_revenue/cleanup.sql
gs://airetail/scripts/daily_product_revenue/compute_daily_product_revenue.sql
gs://airetail/scripts/daily_product_revenue/file_format_converter.sql


In [8]:
!gsutil cp -r ../../apps/daily_product_revenue_bq gs://airetail/apps/daily_product_revenue_bq

Copying file://../../apps/daily_product_revenue_bq/app.py [Content-Type=text/x-python]...
/ [1 files][  653.0 B/  653.0 B]                                                
Operation completed over 1 objects/653.0 B.                                      


In [9]:
!gsutil ls -r gs://airetail/apps/

gs://airetail/apps/:

gs://airetail/apps/daily_product_revenue_bq/:
gs://airetail/apps/daily_product_revenue_bq/app.py


In [1]:
!gcloud config set dataproc/region us-central1

Updated property [dataproc/region].


Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update



In [2]:
!gcloud dataproc workflow-templates list

ID                           JOBS  UPDATE_TIME                  VERSION
getting-started              4     2022-10-08T09:34:41.266501Z  1
wf-daily-product-revenue     4     2022-10-08T11:12:53.276748Z  6
wf-daily-product-revenue-bq  4     2022-10-14T22:13:53.061433Z  6


Here is the command to delete Dataproc Workflow Template (multiline approach doesn't work on Windows)

```shell
gcloud dataproc workflow-templates \
    delete wf-daily-product-revenue-bq
```

In [30]:
!gcloud dataproc workflow-templates delete wf-daily-product-revenue-bq --quiet

Here is the command to create Dataproc Workflow.

```shell
gcloud dataproc workflow-templates \
    create wf-daily-product-revenue-bq
```

In [31]:
!gcloud dataproc workflow-templates create wf-daily-product-revenue-bq

In [32]:
!gcloud dataproc workflow-templates list

ID                           JOBS  UPDATE_TIME                  VERSION
getting-started              4     2022-10-08T09:34:41.266501Z  1
wf-daily-product-revenue     4     2022-10-08T11:12:53.276748Z  6
wf-daily-product-revenue-bq  0     2022-10-16T15:51:28.793690Z  1


Here is the command to attach running or active Dataproc Cluster to the Dataproc Workflow. We need to specify the label for the cluster.

```shell
gcloud dataproc workflow-templates \
    set-cluster-selector \
    wf-daily-product-revenue-bq \
    --cluster-labels goog-dataproc-cluster-name=aidataprocdev
```

In [33]:
!gcloud dataproc workflow-templates set-cluster-selector wf-daily-product-revenue-bq --cluster-labels goog-dataproc-cluster-name=aidataprocdev

Here are the commands to add Spark SQL Jobs to the Dataproc Workflow.

```shell
gcloud dataproc workflow-templates add-job spark-sql \
    --step-id=job-cleanup \
    --file=gs://airetail/scripts/daily_product_revenue/cleanup.sql \
    --workflow-template=wf-daily-product-revenue-bq

# File Format Converter jobs with dependency on cleanup
gcloud dataproc workflow-templates add-job spark-sql \
    --step-id=job-convert-orders \
    --file=gs://airetail/scripts/daily_product_revenue/file_format_converter.sql \
    --params=bucket_name=gs://airetail,table_name=orders \
    --workflow-template=wf-daily-product-revenue-bq \
    --start-after=job-cleanup

gcloud dataproc workflow-templates add-job spark-sql \
    --step-id=job-convert-order-items \
    --file=gs://airetail/scripts/daily_product_revenue/file_format_converter.sql \
    --params=bucket_name=gs://airetail,table_name=order_items \
    --workflow-template=wf-daily-product-revenue-bq \
    --start-after=job-cleanup

# Last Job which depends on convert orders and order_items jobs
gcloud dataproc workflow-templates add-job spark-sql \
    --step-id=job-daily-product-revenue \
    --file=gs://airetail/scripts/daily_product_revenue/compute_daily_product_revenue.sql \
    --params=bucket_name=gs://airetail \
    --workflow-template=wf-daily-product-revenue-bq \
    --start-after=job-convert-orders,job-convert-order-items
```

In [34]:
!gcloud dataproc workflow-templates add-job spark-sql --step-id=job-cleanup --file=gs://airetail/scripts/daily_product_revenue/cleanup.sql --workflow-template=wf-daily-product-revenue-bq

createTime: '2022-10-16T15:51:28.793690Z'
id: wf-daily-product-revenue-bq
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
name: projects/tidy-fort-361710/regions/us-central1/workflowTemplates/wf-daily-product-revenue-bq
placement:
  clusterSelector:
    clusterLabels:
      goog-dataproc-cluster-name: aidataprocdev
updateTime: '2022-10-16T15:51:32.920208Z'
version: 3


In [35]:

!gcloud dataproc workflow-templates add-job spark-sql --step-id=job-convert-orders --file=gs://airetail/scripts/daily_product_revenue/file_format_converter.sql --params=bucket_name=gs://airetail,table_name=orders --workflow-template=wf-daily-product-revenue-bq --start-after=job-cleanup

createTime: '2022-10-16T15:51:28.793690Z'
id: wf-daily-product-revenue-bq
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: orders
  stepId: job-convert-orders
name: projects/tidy-fort-361710/regions/us-central1/workflowTemplates/wf-daily-product-revenue-bq
placement:
  clusterSelector:
    clusterLabels:
      goog-dataproc-cluster-name: aidataprocdev
updateTime: '2022-10-16T15:51:34.450650Z'
version: 4


In [36]:
!gcloud dataproc workflow-templates add-job spark-sql --step-id=job-convert-order-items --file=gs://airetail/scripts/daily_product_revenue/file_format_converter.sql --params=bucket_name=gs://airetail,table_name=order_items --workflow-template=wf-daily-product-revenue-bq --start-after=job-cleanup

createTime: '2022-10-16T15:51:28.793690Z'
id: wf-daily-product-revenue-bq
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: orders
  stepId: job-convert-orders
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: order_items
  stepId: job-convert-order-items
name: projects/tidy-fort-361710/regions/us-central1/workflowTemplates/wf-daily-product-revenue-bq
placement:
  clusterSelector:
    clusterLabels:
      goog-dataproc-cluster-name: aidataprocdev
updateTime: '2022-10-16T15:51:35.794751Z'
version: 5


In [37]:
!gcloud dataproc workflow-templates add-job spark-sql --step-id=job-daily-product-revenue --file=gs://airetail/scripts/daily_product_revenue/compute_daily_product_revenue.sql --params=bucket_name=gs://airetail --workflow-template=wf-daily-product-revenue-bq --start-after=job-convert-orders,job-convert-order-items

createTime: '2022-10-16T15:51:28.793690Z'
id: wf-daily-product-revenue-bq
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: orders
  stepId: job-convert-orders
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: order_items
  stepId: job-convert-order-items
- prerequisiteStepIds:
  - job-convert-orders
  - job-convert-order-items
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/compute_daily_product_revenue.sql
    scriptVariables:
      bucket_name: gs://airetail
  stepId: job-daily-product-revenue
name: projects/tidy-fort

In [38]:
!gcloud dataproc workflow-templates list

ID                           JOBS  UPDATE_TIME                  VERSION
getting-started              4     2022-10-08T09:34:41.266501Z  1
wf-daily-product-revenue     4     2022-10-08T11:12:53.276748Z  6
wf-daily-product-revenue-bq  4     2022-10-16T15:51:37.178034Z  6


* Make sure to specify right project id in below command. Also reformat it to single line while running on Windows.

In [39]:
!gcloud dataproc workflow-templates \
    add-job pyspark \
    --step-id=job-load-dpr-bq \
    --jars=gs://spark-lib/bigquery/spark-bigquery-with-dependencies_2.12-0.26.0.jar \
	--properties=spark.yarn.appMasterEnv.DATA_URI=gs://airetail/retail_gold.db/daily_product_revenue,spark.yarn.appMasterEnv.PROJECT_ID=tidy-fort-361710,spark.yarn.appMasterEnv.DATASET_NAME=retail,spark.yarn.appMasterEnv.GCS_TEMP_BUCKET=airetail \
    --workflow-template=wf-daily-product-revenue-bq \
    --start-after=job-daily-product-revenue \
    gs://airetail/apps/daily_product_revenue_bq/app.py

createTime: '2022-10-16T15:51:28.793690Z'
id: wf-daily-product-revenue-bq
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: orders
  stepId: job-convert-orders
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: order_items
  stepId: job-convert-order-items
- prerequisiteStepIds:
  - job-convert-orders
  - job-convert-order-items
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/compute_daily_product_revenue.sql
    scriptVariables:
      bucket_name: gs://airetail
  stepId: job-daily-product-revenue
- prerequisiteStepIds:
 

In [40]:
!gcloud dataproc workflow-templates describe wf-daily-product-revenue-bq

createTime: '2022-10-16T15:51:28.793690Z'
id: wf-daily-product-revenue-bq
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: orders
  stepId: job-convert-orders
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail
      table_name: order_items
  stepId: job-convert-order-items
- prerequisiteStepIds:
  - job-convert-orders
  - job-convert-order-items
  sparkSqlJob:
    queryFileUri: gs://airetail/scripts/daily_product_revenue/compute_daily_product_revenue.sql
    scriptVariables:
      bucket_name: gs://airetail
  stepId: job-daily-product-revenue
- prerequisiteStepIds:
 

Here is the command to instantiate or run Dataproc Workflow.

```shell
gcloud dataproc workflow-templates \
    instantiate wf-daily-product-revenue-bq
```

In [41]:
!gcloud dataproc clusters list

NAME           PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
aidataprocdev  GCE                                               RUNNING  us-central1-a


In [ ]:
!gcloud dataproc clusters start aidataprocdev

In [ ]:
!gcloud dataproc clusters stop aidataprocdev

In [42]:
# This will take some time to run

!gcloud dataproc workflow-templates instantiate wf-daily-product-revenue-bq

Waiting on operation [projects/tidy-fort-361710/regions/us-central1/operations/54724776-6595-3781-96fa-6aa4cd1f9452].
WorkflowTemplate [wf-daily-product-revenue-bq] RUNNING
Job ID job-cleanup-xn23b5xs2ddue RUNNING
Job ID job-cleanup-xn23b5xs2ddue COMPLETED
Job ID job-convert-orders-xn23b5xs2ddue RUNNING
Job ID job-convert-order-items-xn23b5xs2ddue RUNNING
Job ID job-convert-orders-xn23b5xs2ddue COMPLETED
Job ID job-convert-order-items-xn23b5xs2ddue COMPLETED
Job ID job-daily-product-revenue-xn23b5xs2ddue RUNNING
Job ID job-daily-product-revenue-xn23b5xs2ddue COMPLETED
Job ID job-load-dpr-bq-xn23b5xs2ddue RUNNING
WorkflowTemplate [wf-daily-product-revenue-bq] DONE
Job ID job-load-dpr-bq-xn23b5xs2ddue FAILED
Job ID job-load-dpr-bq-xn23b5xs2ddue error: Google Cloud Dataproc Agent reports job failure. If logs are available, they can be found at:
https://console.cloud.google.com/dataproc/jobs/job-load-dpr-bq-xn23b5xs2ddue?project=tidy-fort-361710&region=us-central1
gcloud dataproc jobs wait